In [3]:
import pyrealsense2 as rs
import numpy as np
import cv2
import pcl
from pcl import pcl_visualization
from plyfile import PlyData, PlyElement
from skimage.draw import polygon

In [4]:
all_points_x = [
    339,
    338,
    337,
    336,
    335,
    334,
    333,
    332,
    331,
    330,
    329,
    328,
    327,
    326,
    325,
    324,
    323,
    322,
    321,
    320,
    319,
    318,
    317,
    316,
    315,
    314,
    313,
    312,
    311,
    310,
    309,
    308,
    307,
    306,
    305,
    304,
    303,
    302,
    301,
    300,
    301,
    302,
    303,
    304,
    305,
    306,
    307,
    308,
    310,
    313,
    315,
    317,
    319,
    321,
    322,
    323,
    324,
    325,
    326,
    327,
    328,
    329,
    330,
    331,
    332,
    334,
    337,
    339,
    341,
    343,
    345,
    348,
    350,
    352,
    354,
    358,
    359,
    360,
    361,
    362,
    363,
    364,
    365,
    366,
    367,
    368,
    369,
    370,
    371,
    372,
    373,
    374,
    375,
    376,
    377,
    378,
    379,
    380,
    378,
    376,
    373,
    371,
    369,
    367,
    364,
    362,
    360,
    358,
    356,
    357,
    358,
    359,
    360,
    361,
    362,
    363,
    364,
    365,
    366,
    367,
    365,
    363,
    361,
    358,
    350
]

all_points_y = [
    263,
    265,
    267,
    270,
    272,
    274,
    276,
    278,
    281,
    283,
    285,
    287,
    289,
    292,
    294,
    296,
    298,
    300,
    303,
    305,
    307,
    309,
    311,
    314,
    316,
    318,
    320,
    322,
    325,
    327,
    329,
    331,
    333,
    335,
    338,
    340,
    342,
    344,
    346,
    350,
    352,
    354,
    357,
    359,
    361,
    363,
    365,
    367,
    368,
    369,
    370,
    371,
    372,
    371,
    369,
    367,
    365,
    363,
    360,
    358,
    356,
    354,
    352,
    350,
    348,
    348,
    349,
    350,
    351,
    352,
    353,
    354,
    355,
    356,
    357,
    356,
    354,
    352,
    350,
    348,
    345,
    343,
    341,
    339,
    337,
    334,
    332,
    330,
    328,
    326,
    323,
    321,
    319,
    317,
    315,
    312,
    310,
    308,
    305,
    304,
    303,
    302,
    301,
    300,
    299,
    298,
    297,
    296,
    295,
    292,
    290,
    288,
    286,
    284,
    281,
    279,
    277,
    275,
    273,
    270,
    269,
    268,
    267,
    266,
    264
]

In [11]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

frames = pipeline.wait_for_frames()
depth_frame = frames.get_depth_frame()
color_frame = frames.get_color_frame()

# img = cv2.imread("./4.png")

# Convert images to numpy arrays
depth_image = np.asanyarray(depth_frame.get_data())
color_image = np.asanyarray(color_frame.get_data())


# width = img.shape[0]
# height = img.shape[1]

width = color_image.shape[0]
height = color_image.shape[1]

mask = np.zeros([width, height, 3], dtype = np.uint8)

rr, cc = polygon(all_points_x, all_points_y)

mask[cc, rr] = color_image[cc, rr]

gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

# # i = 0
# # for row in gray:
# #     not_null = list(filter(lambda x: x != 0, row))
# #     if len(not_null) == 0:
# #         gray = np.delete(gray, i, axis=1)
# #     i += 1

gray = gray[~np.all(gray == 0, axis=1)]
gray = gray.transpose()
gray = gray[~np.all(gray == 0, axis=1)]
gray = gray.transpose()

if

cv2.imshow('test', gray)
cv2.waitKey(0)

pipeline.stop()
cv2.destroyAllWindows()

In [19]:
f = open("verts.txt", "w+")

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

pc = rs.pointcloud()
decimate = rs.decimation_filter()
decimate.set_option(rs.option.filter_magnitude, 2 ** 1)

# Start streaming
pipeline.start(config)

frames = pipeline.wait_for_frames()
depth_frame = frames.get_depth_frame()
color_frame = frames.get_color_frame()

depth_frame = decimate.process(depth_frame)

# img = cv2.imread("./4.png")

# Grab new intrinsics (may be changed by decimation)
depth_intrinsics = rs.video_stream_profile(
    depth_frame.profile).get_intrinsics()
w, h = depth_intrinsics.width, depth_intrinsics.height

# Convert images to numpy arrays
depth_image = np.asanyarray(depth_frame.get_data())
color_image = np.asanyarray(color_frame.get_data())

mapped_frame = depth_frame

points = pc.calculate(depth_frame)
pc.map_to(mapped_frame)

# Pointcloud data to arrays
v, t = points.get_vertices(), points.get_texture_coordinates()
verts = np.asanyarray(v).view(np.float32).reshape(-1, 3)  # xyz
texcoords = np.asanyarray(t).view(np.float32).reshape(-1, 2)  # uv

mapped_frame = v, t

# width = img.shape[0]
# height = img.shape[1]

width = color_image.shape[0]
height = color_image.shape[1]

mask = np.zeros([width, height, 3], dtype = np.uint8)

rr, cc = polygon(all_points_x, all_points_y)

mask[cc, rr] = color_image[cc, rr]

gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

# # i = 0
# # for row in gray:
# #     not_null = list(filter(lambda x: x != 0, row))
# #     if len(not_null) == 0:
# #         gray = np.delete(gray, i, axis=1)
# #     i += 1

gray = gray[~np.all(gray == 0, axis=1)]
gray = gray.transpose()
gray = gray[~np.all(gray == 0, axis=1)]
gray = gray.transpose()

cv2.imshow('test', gray)
cv2.waitKey(0)
print(mapped_frame)
print("cc", rr)

pipeline.stop()
cv2.destroyAllWindows()

TypeError: write() argument must be str, not pyrealsense2.points